In [14]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [15]:
data = pd.read_csv(path+'data/ml_input.csv')

In [16]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [17]:
### Season to test results

N = 2021

In [18]:
model_scores = {
    'model':[],
    'gamma': [],
    'C': [],
    'kernel': [],
    'score': []
    }

In [19]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [20]:
def score_classification(model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        correct_predictions += precision_score(prediction_df.actual, prediction_df.predicted)

    return correct_predictions / df[df.season == N]['round'].nunique()

In [21]:
# Support Vector Machine

params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params.gamma:
    for c in params.C:
        for kernel in params.kernel:
            model_params = (gamma, c, kernel)
            model = svm.SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            model_scores['model'].append('svm_classifier')
            model_scores['gamma'].append(gamma)
            model_scores['C'].append(c)
            model_scores['kernel'].append(kernel)
            model_scores['score'].append(model_score)

In [22]:
comparison_df = pd.DataFrame(model_scores)

comparison_df.sort_values('score', ascending=False)

,model,gamma,C,kernel,score
1598,svm_classifier,0.100000,10.000000,rbf,0.736842
1594,svm_classifier,0.100000,6.951928,rbf,0.736842
1518,svm_classifier,0.069519,10.000000,rbf,0.736842
917,svm_classifier,0.005456,0.263665,poly,0.684211
142,svm_classifier,0.000144,2.335721,rbf,0.684211
...,...,...,...,...,...
1487,svm_classifier,0.069519,0.545559,sigmoid,0.000000
1483,svm_classifier,0.069519,0.379269,sigmoid,0.000000
1479,svm_classifier,0.069519,0.263665,sigmoid,0.000000
1475,svm_classifier,0.069519,0.183298,sigmoid,0.000000


In [23]:
chosen_inputs = comparison_df.query('score > 0.5').sort_values('score', ascending=False)

chosen_inputs.to_csv(path+'parameters/svm_classifier.csv', index=False)